In [1]:
import pandas as pd
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
import pickle as pkl
from keras.optimizers import SGD
import keras.backend as K
from densenet161 import DenseNet 
import numpy as np 
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.models import load_model
import cv2
import numpy as np
from keras.optimizers import SGD
import keras.backend as K
from keras.models import Model
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import pickle as pkl
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)
KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
nb_class=29

In [3]:
def load_data(data_path='data.pkl',label_path='label.pkl'):
    pkl_data_file = open('data.pkl', 'rb')
    pkl_label_file=open('label.pkl','rb')
    data = pkl.load(pkl_data_file)
    label=pkl.load(pkl_label_file)
    return data,label

In [4]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
	image_paths = sorted(list(paths.list_images(p_path)))
	data=[]
	image_name=[]
	for im_path in image_paths:
		image=cv2.imread(im_path)
		image=cv2.resize(image,(224,224))
		image = img_to_array(image)
		image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
		image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
		image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
		data.append(image)
		image_name.append(im_path.split('/')[2])
	data = np.array(data, dtype="float32")#transfer to numpy
	model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
	sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
	model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
	out= model.predict(data)
	label=np.argmax(out[0])
	print(type(label))
	print(label)
	print(label.shape)
	predict_label = pd.read_csv("data/submit_example.csv")
	for i in range(predict_label.values.shape[0]):
		find_name=predict_label.values[i][0]
		for j in range(data.shape[0]):
			if find_name==image_name[j]:
				predict_label.iat[i,1]=np.argmax(out[j])+1
	predict_label.to_csv("data/predict.csv",index=False)

In [5]:

def load_new_model(weight_path='imagenet_models/densenet161_weights_tf.h5'):
	base_model = DenseNet(reduction=0.5, classes=1000, weights_path=weight_path)
	# model = Model(input=base_model.input, output=base_model.get_layer('conv5_blk_scale').output)
	x=base_model.get_layer('conv5_blk_scale').output
	x = Activation('relu', name='relu_last')(x)
	x=GlobalAveragePooling2D(name='pool_last')(x)
	x= Dense(nb_class, name='fc_last')(x)
	x=Activation('softmax', name='prob_last')(x)
	model=Model(input=base_model.input,output=x,name='new_densenet')
	for layer in model.layers[:-4]:
	    layer.trainable = False
	return model



In [6]:
def train_data(data,label,model):
    (trainX, testX, trainY, testY) = train_test_split(data,
        label, test_size=0.2, random_state=42)
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, rotation_range=20,
                                 width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    print(trainX.shape[0])
    batchs=50
    model.fit_generator(datagen.flow(trainX, trainY, batch_size=batchs),steps_per_epoch=trainX.shape[0]//batchs, 
                        epochs=5, verbose=1)
#     model.fit(trainX,trainY,batch_size=100,epochs=5,verbose=1,validation_data=(testX,testY))
    model.save_weights('imagenet_models/new_dense_weight.h5')
    model.save('imagenet_models/new_densenet.h5')
    score = model.evaluate(testX,testY,verbose=0)
    print ('accuracy:{}'.format(score[1]))

In [7]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
    image_paths = sorted(list(paths.list_images(p_path)))
    data=[]
    image_name=[]
    for im_path in image_paths:
        image=cv2.imread(im_path)
        image=cv2.resize(image,(224,224))
        image = img_to_array(image)
        image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
        image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
        image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
        data.append(image)
        image_name.append(im_path.split('/')[2])
    data = np.array(data, dtype="float32")#transfer to numpy
    model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
    #origin ls is 1e-2
    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    out= model.predict(data)
    return out
# 	label=np.argmax(out[0])
# 	print(type(label))
# 	print(label)
# 	print(label.shape)
# 	predict_label = pd.read_csv("data/submit_example.csv")
# 	for i in range(predict_label.values.shape[0]):
# 		find_name=predict_label.values[i][0]
# 		for j in range(data.shape[0]):
# 			if find_name==image_name[j]:
# 				predict_label.iat[i,1]=np.argmax(out[j])+1
# 	predict_label.to_csv("data/predict.csv",index=False)

In [8]:
# load data then train data
data,label=load_data()
print(len(label))
label_out=np.zeros((len(label),nb_class))
                
for i in range(len(label)):
    t=0
    for j in range(len(label[i])):
        tmp=label[i][j]
        tmp=np.array(tmp)
        tmp=np_utils.to_categorical(tmp,nb_class)
        t=t+tmp
        label_out[i]=t
data_out = np.array(data, dtype="float32")
model=load_new_model()
train_data(data_out,label_out,model)

10665


/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), name="conv1", strides=(2, 2), use_bias=False)`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv3_12_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv3_12_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:136: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(384, (1, 1), name="conv3_blk", use_bias=False)`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update y

/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_19_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_19_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_20_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_20_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D`

/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_2_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_2_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_3_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_3_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` cal

/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_21_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_21_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_22_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_22_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D`

8532
Epoch 1/5


/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:505: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras/preprocessing/image.py:513: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


170/170 [==============================] - 138s - loss: 2.2854 - acc: 0.3584   
Epoch 2/5
170/170 [==============================] - 61s - loss: 1.9921 - acc: 0.4198    
Epoch 3/5
170/170 [==============================] - 61s - loss: 1.9132 - acc: 0.4308    
Epoch 4/5
170/170 [==============================] - 62s - loss: 1.8757 - acc: 0.4398    
Epoch 5/5
170/170 [==============================] - 62s - loss: 1.8388 - acc: 0.4434    
accuracy:0.4149085796192337


In [9]:
#predict data
out=predict_data()

/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), name="conv1", strides=(2, 2), use_bias=False)`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv3_11_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv3_11_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv3_12_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv3_12_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:136: UserWarning: Update your `Conv2D

/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_17_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_18_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_18_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_19_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D

/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_35_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_36_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_36_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:136: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1056, (1, 1), name="conv4_blk", use_bias=False)`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your

/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_17_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_18_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_18_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_19_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D

In [10]:
p_path="data/test"
image_paths = sorted(list(paths.list_images(p_path)))
image_name=[]
for im_path in image_paths:
    image_name.append(im_path.split('/')[2])
predict_label = pd.read_csv("data/submit_example.csv")
for i in range(predict_label.values.shape[0]):
    find_name=predict_label.values[i][0]
    for j in range(len(image_name)):
        if find_name==image_name[j]:
            tmp_out=out[j]
            list_t=[]
            for m in range(tmp_out.shape[0]):
                if tmp_out[m]>0.2:
                    list_t.append(m+1)
            if len(list_t)==0:
                print(image_name[j])
                list_t.append(np.argmax(tmp_out))
            str_s=""
            for n in range(len(list_t)):
                str_s=str_s+str(list_t[n])
                if n!=len(list_t)-1:
                    str_s=str_s+";"
            predict_label.iloc[i,1]=str_s
predict_label.to_csv("data/predict.csv",index=False)

0081e91a5b284302a944a24e3f936d26.jpg
012f541f67164de9994eb6bdaf78bad9.jpg
0151feac33e249e983aee93003d417ca.jpg
03666a8253774184b32a81f58e42f632.png
052dbe36e1b9428e95a58a7aca7d7e6f.jpg
06d36e1d0b474ac09398473dbebeff08.jpg
08c714ff2e674a79b27d4bb7cd670c77.jpg
0b381369f20543eda523445aa6027686.jpg
0b54ea8250fd4cdcb3b14e2ce1fd7e36.jpg
0cd2a02b115d4d13a0e403c84769896c.jpg
0f24b9d593e7413383c63424212ac41f.jpg
1048bdb74975412fa6e73000aac95d11.png
106af1dc886c4a84855faae7ce4130bf.jpg
109b1cee81a54b17b420a8acd6846834.jpg
1165c22310d14374b4570a73e8011334.jpg
12cdcf01632648f7bf6d785be67b6638.jpg
14b77b330f794393b9aa61c69cf27a88.jpg
14ca266d87464388a6222a3663b9ce92.png
158f25a387d34108aa0b488b973e7520.jpg
1639176450d541f9ad7a6e054d895e76.jpg
1698dd9508df40b5b79ccc9ae16b4c7f.png
17092994a8994cacbb2726d729a00ded.jpg
1a3a55e3bc0f488cbff713b110dbebcc.png
1a3d5ad5836b41189072341ddd0aa837.jpg
1bc712dd9e4444319ceacd2117c6470d.jpg
1cd03a77621b4b7bb8ca55bf93548c62.png
1e0cc495150f41e3a01a208841e5c27d.png
1